In [30]:
const three = await import("https://cdn.jsdelivr.net/npm/three@0.174.0/+esm");

Graph = class Graph {
  #scene = null;
  #skyColor = null;
  #groundColor = null;
  #intensity = null;
  #light = null;
  #camera = null;
  #renderer = null;
  #vertices = null;
  #edges = null;
  
  constructor( elem ){
    // using the threejs library to create a scene
    this.#scene = new three.Scene();
    
    // light
    this.#skyColor = 0xB1E1FF;
    this.#groundColor = 0xB97A20;
    this.#intensity = 1;
    this.#light = new three.HemisphereLight( 
      this.#skyColor, 
      this.#groundColor, 
      this.#intensity 
    );
    this.#scene.add( this.#light );
    
    // the camera lets us look into the scene
    this.#camera = new three.PerspectiveCamera( 75, 500/350, 0.1, 1000 );
    this.#camera.position.z = 15;
    
    // the renderer renders the scene onto a canvas element
    this.#renderer = new three.WebGLRenderer({ antialias: true });
    this.#renderer.setSize( 500, 350 );
    this.#renderer.setClearColor( 'white' );
    this.#renderer.domElement.style.border = '1px dashed orangered';
    this.#renderer.setAnimationLoop( this.#animate.bind( this ) );
    
    // todo
    elem.appendChild( this.#renderer.domElement )

    this.#vertices = new Set();
    this.#edges = new Set();
  }

  #animate(){
    this.#renderer.render( this.#scene, this.#camera )

    this.#vertices.values().forEach( vertex => vertex.update() );
    this.#edges.values().forEach( edge => edge.update() );
  }

  addVertex( vertex ){
    this.#scene.add( vertex.group );
    this.#vertices.add( vertex );
  }

  removeVertex( vertex ){
    this.#scene.remove( vertex.group )
    for(const edge of this.#edges.values()){
      if(edge.source === vertex || edge.target === vertex){
        edge.remove();
      }
    }
  }

  addEdge( edge ){
    this.#scene.add( edge.group );
    this.#edges.add( edge );
  }

  removeEdge( edge ){
    this.#scene.remove(edge.group);
    this.#edges.delete(edge);
  }
}

Vertex = class Vertex {
  #graph = null;
  #cube = null;
  #wire = null;
  #group = null;

  #position = null;
  
  constructor( graph ){
    this.#graph = graph;
    
    // creating the objects visible in the scene
    // inside a little blue cube
    this.#cube = new three.Mesh(
      new three.BoxGeometry( 1, 1, 1 ),
      new three.MeshPhongMaterial({ color: 'orangered' })
    );
    this.#wire = new three.Mesh( 
      new three.BoxGeometry( 1.25, 1.25, 1.25 ), 
      new three.MeshPhongMaterial({ color: 0x00beef, wireframe: true })
    );
    this.#wire.visible = false;

    this.#group = new three.Group();
    this.#group.add(this.#cube);
    this.#group.add(this.#wire);
    
    // add group to scene
    this.#graph.addVertex(this);
  }

  get group(){
    return this.#group;
  }

  get color(){
    return this.#cube.material.color;
  }
  
  set color( value ){
    this.#cube.material.color.set( value );
  }

  get selectionColor(){
    return this.#wire.material.color;
  }

  set selectionColor( value ){
    this.#wire.material.color.set( value );
  }

  set size( value ){
    this.#group.scale.set( value, value, value );
  }

  get selected(){
    return this.#wire.visible;
  }
  
  set selected( value ){
    this.#wire.visible = value;
  }
  
  get pos(){
    return this.#group.position;
  }
  
  set pos( { x, y, z } ){
    this.#group.position.set( x, y, z );
  }

  update(){
    this.#group.rotation.x += 0.01;
    this.#group.rotation.y += 0.01;
  }

  remove(){
    this.#graph.removeVertex(this);
  }

  // aliases
  get position(){
    return this.pos;
  }
  
  set position({ x, y, z }){
    this.pos = { x, y, z };
  }
  
}

Edge = class Edge {
  #graph = null;
  #line = null;
  #group = null;
  
  #source = null;
  #target = null;

  constructor( graph, source, target ){
    this.#graph = graph;
    this.#source = source;
    this.#target = target;
  
    this.#line = new three.Line(
      new three.BufferGeometry().setFromPoints( [ this.#source.pos, this.#target.pos ] ),
      new three.LineBasicMaterial( { color: 0x0000ff } )
    );

    this.#group = new three.Group();
    this.#group.add(this.#line);

    this.#graph.addEdge(this);
  }

  get group(){
    return this.#group;
  }

  get source(){
    return this.#source;
  }

  get target(){
    return this.#target;
  }

  update(){
    this.setOrigin(this.#source.pos);
    this.setEnd(this.#target.pos);
  }

  setOrigin( { x, y, z } ){
    const positionAttr = this.#line.geometry.getAttribute('position');
    positionAttr.setXYZ( 0, x, y, z );
    positionAttr.needsUpdate = true;
  }

  setEnd( { x, y, z } ){
    const positionAttr = this.#line.geometry.getAttribute('position');
    positionAttr.setXYZ( 1, x, y, z );
    positionAttr.needsUpdate = true;
  }

  remove(){
    this.#graph.removeEdge(this);
  }

  set color(value){
    this.#line.material.color.set(value);
  }
}

randomPos = () => {
  return new three.Vector3( 
    Math.random() * 5,
    Math.random() * 5,
    Math.random() * 5
  )
}

graph = new Graph( output );

vertex = new Vertex( graph );
vertex.position = randomPos();

vertex2 = new Vertex( graph );
vertex2.position = randomPos();

edge = new Edge( graph, vertex, vertex2 );

<canvas data-engine="three.js r174" width="500" height="350" style="display: block; width: 500px; height: 350px; border: 1px dashed orangered;"></canvas>